# Preparing the MIMIC 3 dataset

MIMIC3 can be downloaded from [here](https://mimic.physionet.org/gettingstarted/access/).

We use the same preprocessing as the authors of the GRU-ODE-Bayer paper and provide the final data preparation notebook that is missing in the original repository. First, execute the notebooks *admissions*, *outputs*, *labevents* and *prescriptions* provided [here](https://github.com/edebrouwer/gru_ode_bayes/tree/master/data_preproc/MIMIC).

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import numpy as np

In [44]:
file_path="C:/Users/rswfa/Documents/Github/neural-flows-experiments/nfe/experiments/data/Data/"
outfile_path="C:/Users/rswfa/Documents/Github/neural-flows-experiments/nfe/experiments/data/Data/"
lab_df=pd.read_csv(file_path+"LAB_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","VALUENUM","LABEL"]]
inputs_df=pd.read_csv(file_path+"INPUTS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","AMOUNT","LABEL"]]
outputs_df=pd.read_csv(file_path+"OUTPUTS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","VALUE","LABEL"]]
presc_df=pd.read_csv(file_path+"PRESCRIPTIONS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","DOSE_VAL_RX","DRUG"]]

In [45]:
inputs_df["VALUENUM"]=inputs_df["AMOUNT"]
inputs_df.head()
inputs_df=inputs_df.drop(columns=["AMOUNT"]).copy()

outputs_df["VALUENUM"]=outputs_df["VALUE"]
outputs_df=outputs_df.drop(columns=["VALUE"]).copy()

presc_df["VALUENUM"]=presc_df["DOSE_VAL_RX"]
presc_df=presc_df.drop(columns=["DOSE_VAL_RX"]).copy()
presc_df["LABEL"]=presc_df["DRUG"]
presc_df=presc_df.drop(columns=["DRUG"]).copy()

inputs_df["Origin"]="Inputs"
lab_df["Origin"]="Lab"
outputs_df["Origin"]="Outputs"
presc_df["Origin"]="Prescriptions"

merged_df1=(inputs_df.append(lab_df)).reset_index()
merged_df2=(merged_df1.append(outputs_df)).reset_index()
merged_df2.drop(columns="level_0",inplace=True)
merged_df=(merged_df2.append(presc_df)).reset_index()

assert(merged_df["LABEL"].nunique()==(inputs_df["LABEL"].nunique()+lab_df["LABEL"].nunique()+outputs_df["LABEL"].nunique()+presc_df["LABEL"].nunique()))

In [46]:
merged_df.head()

,level_0,index,SUBJECT_ID,HADM_ID,CHARTTIME,LABEL,VALUENUM,Origin
0,0,0.0,42281,195911.0,2119-10-17 20:02:00,K Phos,1.225762,Inputs
1,1,1.0,42281,195911.0,2119-10-17 20:32:00,K Phos,1.225762,Inputs
2,2,2.0,42281,195911.0,2119-10-17 21:02:00,K Phos,1.225762,Inputs
3,3,3.0,42281,195911.0,2119-10-17 21:32:00,K Phos,1.225762,Inputs
4,4,4.0,42281,195911.0,2119-10-17 23:08:00,K Phos,1.121884,Inputs


In [47]:
merged_df['CHARTTIME']=pd.to_datetime(merged_df["CHARTTIME"], format='%Y-%m-%d %H:%M:%S')
ref_time=merged_df.groupby("HADM_ID")["CHARTTIME"].min()

merged_df_1=pd.merge(ref_time.to_frame(name="REF_TIME"),merged_df,left_index=True,right_on="HADM_ID")
merged_df_1["TIME_STAMP"]=merged_df_1["CHARTTIME"]-merged_df_1["REF_TIME"]
assert(len(merged_df_1.loc[merged_df_1["TIME_STAMP"]<timedelta(hours=0)].index)==0)

In [48]:
label_dict=dict(zip(list(merged_df_1["LABEL"].unique()),range(len(list(merged_df_1["LABEL"].unique())))))
merged_df_1["LABEL_CODE"]=merged_df_1["LABEL"].map(label_dict)

merged_df_short=merged_df_1[["HADM_ID","VALUENUM","TIME_STAMP","LABEL_CODE","Origin"]]

In [49]:
label_dict_df=pd.Series(merged_df_1["LABEL"].unique()).reset_index()
label_dict_df.columns=["index","LABEL"]
label_dict_df["LABEL_CODE"]=label_dict_df["LABEL"].map(label_dict)
label_dict_df.drop(columns=["index"],inplace=True)
label_dict_df.to_csv(outfile_path+"label_dict.csv")

In [50]:
merged_df_short["VALUENUM"] = merged_df_short["VALUENUM"].astype(float)

<ipython-input-50-88889de71fc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_short["VALUENUM"] = merged_df_short["VALUENUM"].astype(float)


In [51]:
# merged_df_short=merged_df_short.loc[(merged_df_short["TIME_STAMP"]<2880)]
print("Number of patients considered :"+str(merged_df_short["HADM_ID"].nunique()))

Number of patients considered :58


In [52]:
merged_df_short.drop(["Origin"], axis=1, inplace=True)
merged_df_short

C:\Users\rswfa\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,HADM_ID,VALUENUM,TIME_STAMP,LABEL_CODE
11417,100375.0,1.1,0 days 02:30:00,0
11586,100375.0,16.0,0 days 00:00:00,1
11732,100375.0,3.1,0 days 00:00:00,2
11733,100375.0,3.0,1 days 04:09:00,2
11854,100375.0,99.0,0 days 00:00:00,3
...,...,...,...,...
32715,199395.0,10.0,5 days 00:00:00,41
32716,199395.0,100.0,0 days 00:00:00,40
32717,199395.0,40.0,3 days 00:00:00,39
32718,199395.0,20.0,5 days 00:00:00,41


In [53]:
# merged_df_short = complete_df
complete_df = merged_df_short.copy()
labels = complete_df["LABEL_CODE"].unique()
# labels = merged_df_short["LABEL_CODE"].unique()
value_columns = []
mask_columns  = []
for num in labels:
    name = "Value_label_" + str(num)
    name2 = "Mask_label_" + str(num)
    value_columns.append(name)
    mask_columns.append(name2)
    complete_df[name] = 0
    complete_df[name2] = 0
    complete_df[name] = complete_df[name].astype(float)

In [55]:
complete_df.dropna(inplace=True)
for index, row in complete_df.iterrows():
    name = "Value_label_" + str(row["LABEL_CODE"])
    name2 = "Mask_label_" + str(row["LABEL_CODE"])
    complete_df.at[index, name] = row["VALUENUM"]
    complete_df.at[index, name2] = 1

In [56]:
complete_df.drop(["VALUENUM", "LABEL_CODE"], axis=1, inplace=True)
complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()
for x in mask_columns:
    assert(len(complete_df.loc[complete_df[x]>1])==0)
complete_df

,HADM_ID,TIME_STAMP,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,Mask_label_3,...,Value_label_81,Mask_label_81,Value_label_82,Mask_label_82,Value_label_83,Mask_label_83,Value_label_84,Mask_label_84,Value_label_85,Mask_label_85
0,100375.0,0 days 00:00:00,0.0,0,16.0,1,3.1,1,99.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,100375.0,0 days 00:39:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,100375.0,0 days 02:30:00,1.1,1,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,100375.0,0 days 09:39:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,100375.0,0 days 10:10:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10154,199395.0,9 days 13:35:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
10155,199395.0,9 days 15:04:00,0.0,0,38.0,1,0.0,0,44.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
10156,199395.0,10 days 10:10:00,0.0,0,51.0,1,0.0,0,104.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
10157,199395.0,11 days 05:56:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [58]:
complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()
complete_df

,HADM_ID,TIME_STAMP,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,Mask_label_3,...,Value_label_81,Mask_label_81,Value_label_82,Mask_label_82,Value_label_83,Mask_label_83,Value_label_84,Mask_label_84,Value_label_85,Mask_label_85
0,100375.0,0 days 00:00:00,0.0,0,16.0,1,3.1,1,99.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,100375.0,0 days 00:39:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,100375.0,0 days 02:30:00,1.1,1,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,100375.0,0 days 09:39:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,100375.0,0 days 10:10:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10154,199395.0,9 days 13:35:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
10155,199395.0,9 days 15:04:00,0.0,0,38.0,1,0.0,0,44.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
10156,199395.0,10 days 10:10:00,0.0,0,51.0,1,0.0,0,104.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
10157,199395.0,11 days 05:56:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [60]:
unique_ids=np.arange(complete_df["HADM_ID"].nunique())
np.random.shuffle(unique_ids)
d=dict(zip(complete_df["HADM_ID"].unique(),unique_ids))

Unique_id_dict=pd.Series(complete_df["HADM_ID"].unique()).reset_index().copy()
Unique_id_dict.columns=["index","HADM_ID"]
Unique_id_dict["unique_id"]=Unique_id_dict["HADM_ID"].map(d)
Unique_id_dict.to_csv("unique_id_dict.csv")

unique_id_df = pd.read_csv("unique_id_dict.csv")
d = dict(zip(unique_id_df["HADM_ID"].values,unique_id_df["unique_id"].values))

In [65]:
admissions=pd.read_csv(file_path+"admissions_processed.csv")
death_tags_s=admissions.groupby("HADM_ID")["DEATHTAG"].unique().astype(int).to_frame().reset_index()
death_tags_df=death_tags_s.loc[death_tags_s["HADM_ID"].isin(complete_df["HADM_ID"])].copy()
death_tags_df["unique_id"]=death_tags_df["HADM_ID"].map(d)
death_tags_df.sort_values(by="unique_id",inplace=True)
death_tags_df.rename(columns={"DEATHTAG":"Value"},inplace=True)
death_tags_df.set_index("unique_id", inplace=True)
death_tags_df.drop("HADM_ID", axis=1, inplace=True)
death_tags_df.to_csv("complete_death_tags.csv")

complete_df["unique_id"] = complete_df["HADM_ID"].map(d)

In [67]:
complete_df.rename(columns={"unique_id": "ID", "time_stamp": "Time"}, inplace=True)
complete_df.drop(["HADM_ID"], axis=1, inplace=True)
complete_df.set_index(["ID"], inplace=True)
complete_df

,TIME_STAMP,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,Mask_label_3,Value_label_4,...,Value_label_81,Mask_label_81,Value_label_82,Mask_label_82,Value_label_83,Mask_label_83,Value_label_84,Mask_label_84,Value_label_85,Mask_label_85
ID,,,,,,,,,,,,,,,,,,,,,
13,0 days 00:00:00,0.0,0,16.0,1,3.1,1,99.0,1,14.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
13,0 days 00:39:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
13,0 days 02:30:00,1.1,1,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
13,0 days 09:39:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
13,0 days 10:10:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,9 days 13:35:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
41,9 days 15:04:00,0.0,0,38.0,1,0.0,0,44.0,1,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
41,10 days 10:10:00,0.0,0,51.0,1,0.0,0,104.0,1,14.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [ ]:
complete_df.to_csv("full_dataset.csv")